In [59]:
!python preprocess.py

In [13]:
!mkdir THUCNews/saved_dict

In [15]:
!python run.py --model bert --save_path bert_predictions.npz

Namespace(model='bert', save_path='bert_predictions.npz')
bert
Loading data...
98529it [00:09, 10757.29it/s]
33267it [00:03, 10405.47it/s]
32935it [00:03, 10472.53it/s]
Time usage: 0:00:16
Epoch [1/3]
/paddle/imcs21/task/DAC/BERT-DAC/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Iter:      0,  Val P: 0.04433,  Val R: 5.9731%,  Val F1: 0.02457,  Val Acc: 6.0120%,  Time: 0:01:03 *
/usr/local/lib/python3.7/dist-p

In [60]:
!python run.py --model bert --save_path bert_predictions1.npz

Namespace(model='bert', save_path='bert_predictions1.npz')
bert
Loading data...
98529it [00:08, 10972.61it/s]
33267it [00:03, 10734.60it/s]
25310it [00:02, 10624.27it/s]
Time usage: 0:00:14
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` param

# predict

In [53]:
import numpy as np
import pandas as pd

In [54]:
import json
import os


def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data


data_dir = '../../../dataset'

In [61]:
a = np.load('bert_predictions1.npz')

In [34]:
# a.files # ['test_confusion', 'test_prediction']

In [35]:
# a[a.files[1]] # array([13, 14, 14, ...,  7, 15, 15])

In [62]:
r = pd.Series(a[a.files[1]])

In [63]:
r.shape

(25310,)

In [64]:
r.head()

0    15
1    14
2    13
3    14
4    14
dtype: int64

In [65]:
r1 = r.map(id2tag)

In [66]:
r1.head()

0              Other
1     Inform-Symptom
2    Request-Symptom
3     Inform-Symptom
4     Inform-Symptom
dtype: object

In [67]:
test_set = load_json(os.path.join(data_dir, 'test.json'))

In [40]:
# test_set
# {'10035922': {'self_report': '小孩5岁，从6月至今，总是间接性咳嗽，偶尔有痰，不思饮食，中间吃过头孢克洛，阿莫西林，止咳糖浆，盐酸溴索口服液，利巴韦林喷剂等药，总是时好时犯，请问怎么办，谢谢医生',
#   'explicit_info': {'Symptom': ['咳嗽', '痰']},
#   'dialogue': [{'sentence_id': '1',
#     'speaker': '医生',
#     'sentence': '这个孩子除了咳嗽以外，还有其他的症状吗？'},
#    {'sentence_id': '2', 'speaker': '患者', 'sentence': '前1周有脖子处淋巴结肿大'},
#    {'sentence_id': '3', 'speaker': '患者', 'sentence': '发烧'},
#    {'sentence_id': '4', 'speaker': '医生', 'sentence': '这个孩子断断续续的咳嗽有多长时间了？'},
#    {'sentence_id': '5', 'speaker': '患者', 'sentence': '7月份至今'},
#    {'sentence_id': '6', 'speaker': '医生', 'sentence': '你都进行过什么检查？'},

In [68]:
tags = [
    'Request-Etiology', 'Request-Precautions', 'Request-Medical_Advice', 'Inform-Etiology', 'Diagnose',
    'Request-Basic_Information', 'Request-Drug_Recommendation', 'Inform-Medical_Advice',
    'Request-Existing_Examination_and_Treatment', 'Inform-Basic_Information', 'Inform-Precautions',
    'Inform-Existing_Examination_and_Treatment', 'Inform-Drug_Recommendation', 'Request-Symptom',
    'Inform-Symptom', 'Other'
]
id2tag = {idx:tag for idx, tag in enumerate(tags)}

In [69]:
rd = {}
i = 0
for pid, sample in test_set.items():
    td = {}
    for sent in sample['dialogue']:
        td[sent['sentence_id']] = r1.iloc[i]
        
        i += 1
    rd[pid] = td
#     break

In [70]:
import json
def write2json_file(fp, data, encoding='utf-8'):
    """
    Converts arbitrary object recursively into JSON file. 
    Use ensure_ascii=false to output UTF-8.
    """
    with open(fp, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

In [71]:
write2json_file('IMCS-IR_test.json', rd)